In [2]:
%load_ext autoreload
%autoreload 2

In [23]:
import json
from project.models import Project
from public_data.models import Ocsge2015, CouvertureSol, UsageSol
from prettytable import PrettyTable
from random import randint

In [9]:
raw_data = Project.objects.get(pk=4).couverture_usage
raw_data

{'2015': {'usage': {'US2': 1.893910588710199,
   'US3': 10.807377105104973,
   'US5': 42.89614451715585,
   'US1.1': 34.43719787344733,
   'US1.2': 331.8675996277132,
   'US1.3': 1.0089925113720215,
   'US1.4': 0.10564807648016929,
   'US235': 2.6416211796457225,
   'US4.3': 5.711383164151101,
   'US6.1': 0.9764485456040662,
   'US6.2': 0.06821666027812875,
   'US6.3': 9.707919245245414,
   'US4.1.1': 9.485539593656883,
   'US4.1.2': 0.4050821703559742,
   'US4.1.3': 0.19064125108183277,
   'US4.1.4': 0.010462694701598258},
  'couverture': {'CS1.2.1': 0.06687055043883208,
   'CS1.2.2': 2.6962689281887404,
   'CS2.1.1': 5.447071515751622,
   'CS2.1.2': 3.669919657128981,
   'CS2.2.1': 76.67959574988951,
   'CS1.1.1.1': 14.270858973249133,
   'CS1.1.1.2': 9.330707903868083,
   'CS1.1.2.1': 4.433014098170347,
   'CS1.1.2.2': 0.03908845978729424,
   'CS2.1.1.1': 45.589656538816484,
   'CS2.1.1.2': 258.89947799131437,
   'CS2.1.1.3': 31.091654438100917}},
 '2018': {'usage': {'US2': 2.252283

In [24]:

raw_data

{'2015': {'couverture': {'CS1': 1,
   'CS2': 1,
   'CS1.1': 0,
   'CS1.1.1': 0,
   'CS1.1.1.1': 3,
   'CS1.1.1.2': 2,
   'CS1.1.2': 1,
   'CS1.1.2.1': 1,
   'CS1.1.2.2': 2,
   'CS1.2': 3,
   'CS1.2.1': 1,
   'CS1.2.2': 2,
   'CS1.2.3': 1,
   'CS2.1': 3,
   'CS2.1.1': 0,
   'CS2.1.1.1': 3,
   'CS2.1.1.2': 2,
   'CS2.1.1.3': 0,
   'CS2.1.2': 0,
   'CS2.1.3': 0,
   'CS2.2': 2,
   'CS2.2.1': 3,
   'CS2.2.2': 3},
  'usage': {'US1': 1,
   'US2': 1,
   'US3': 3,
   'US4': 2,
   'US5': 3,
   'US6': 3,
   'US1.3': 0,
   'US1.4': 1,
   'US1.5': 3,
   'US235': 2,
   'US4.1': 1,
   'US4.2': 2,
   'US4.3': 1,
   'US6.1': 2,
   'US6.2': 0,
   'US6.3': 3,
   'US6.6': 2,
   'US1.1': 1,
   'US4.1.1': 1,
   'US4.1.2': 0,
   'US4.1.4': 2,
   'US4.1.5': 0,
   'US4.1.3': 2,
   'US1.2': 2}},
 '2018': {'couverture': {'CS1': 1,
   'CS2': 1,
   'CS1.1': 1,
   'CS1.1.1': 3,
   'CS1.1.1.1': 3,
   'CS1.1.1.2': 3,
   'CS1.1.2': 2,
   'CS1.1.2.1': 2,
   'CS1.1.2.2': 3,
   'CS1.2': 2,
   'CS1.2.1': 2,
   'CS1.2.2': 

In [28]:
# test du code qui permet d'évaluer chaque node de CouvertureSol ou UsageSol

couv_codes = CouvertureSol.objects.all().values_list("code_prefix", flat=True)
usage_codes = UsageSol.objects.all().values_list("code_prefix", flat=True)
for year in ["2015", "2018"]:
    raw_data[year] = {
        "couverture": {code: randint(0,3) for code in couv_codes},
        "usage": {code: randint(0,3) for code in usage_codes},
    }

millesimes = raw_data.keys()

headers = ["Code", "Libellé", "raw data 2015", "2015", "raw data 2018", "2018"]

couv_table = PrettyTable(field_names=headers, align="l")
couv_table.add_rows(
    (
        _.code_prefix,
        _.label[:30],
        raw_data['2015']["couverture"][_.code_prefix],
        _.total_surface["2015"],
        raw_data['2018']["couverture"][_.code_prefix],
        _.total_surface["2018"]
    ) 
    for _ in CouvertureSol.iter_total_surface(raw_data)
)

usage_table = PrettyTable(field_names=headers, align="l")
usage_table.add_rows(
    (
        _.code_prefix,
        _.label[:30],
        raw_data['2015']["usage"][_.code_prefix],
        _.total_surface["2015"],
        raw_data['2018']["usage"][_.code_prefix],
        _.total_surface["2018"]
    ) 
    for _ in UsageSol.iter_total_surface(raw_data)
)

print("Couverture:")
print(couv_table)
print("Usage:")
print(usage_table)

Couverture:
+-----------+--------------------------------+---------------+------+---------------+------+
| Code      | Libellé                        | raw data 2015 | 2015 | raw data 2018 | 2018 |
+-----------+--------------------------------+---------------+------+---------------+------+
| CS1       | Sans végétation                | 1             | 17   | 1             | 24   |
| CS1.1     | Surfaces anthropisées          | 0             | 9    | 1             | 17   |
| CS1.1.1   | Zones imperméables             | 0             | 5    | 3             | 9    |
| CS1.1.1.1 | Zones bâties                   | 3             | 3    | 3             | 3    |
| CS1.1.1.2 | Zones non bâties (Routes; plac | 2             | 2    | 3             | 3    |
| CS1.1.2   | Zones perméables               | 1             | 4    | 2             | 7    |
| CS1.1.2.1 | Zones à matériaux minéraux     | 1             | 1    | 2             | 2    |
| CS1.1.2.2 | Zones à autres matériaux compo | 2          